# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv("..\starter_code\city_data.csv")
city_weather_df


,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Oistins,13.0667,-59.5333,77.09,83,40,13.80,BB,1643697694
1,Cape Town,-33.9258,18.4232,76.10,69,0,1.99,ZA,1643697546
2,Sampit,-2.5333,112.9500,94.08,42,71,3.38,ID,1643697695
3,Barrow,71.2906,-156.7887,-12.98,77,100,5.75,US,1643697695
4,Creel,27.7500,-107.6333,33.01,64,44,2.59,MX,1643697696
...,...,...,...,...,...,...,...,...,...
572,Pochinok,54.4068,32.4407,21.16,95,99,9.28,RU,1643698051
573,Gamba,-2.6500,10.0000,78.48,84,49,2.06,GA,1643698051
574,Pangnirtung,66.1451,-65.7125,-27.65,97,100,4.18,CA,1643698052
575,Moree,-29.4667,149.8500,96.82,26,99,5.75,AU,1643697782


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations
locations = city_weather_df[["Lat","Lng"]]

#Add Heatmap layer to map
humidity = city_weather_df["Humidity"]

In [4]:
#Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
smaller_city_df = city_weather_df.loc[(city_weather_df["Wind Speed"] <= 10) & (city_weather_df["Cloudiness"]== 0) & \
                  (city_weather_df["Max temp"] >= 70) & (city_weather_df["Max temp"] <= 80)].dropna()
smaller_city_df

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,Cape Town,-33.9258,18.4232,76.10,69,0,1.99,ZA,1643697546
52,Makakilo City,21.3469,-158.0858,71.92,74,0,0.00,US,1643697726
59,Kapaa,22.0752,-159.3190,73.38,70,0,4.61,US,1643697730
83,East London,-33.0153,27.9116,77.94,78,0,4.61,ZA,1643697746
129,Praia,14.9215,-23.5087,70.34,83,0,3.44,CV,1643697771
230,Keti Bandar,24.1447,67.4497,78.80,46,0,2.17,PK,1643697838
231,Pochutla,15.7432,-96.4661,76.82,85,0,5.21,MX,1643697838
296,Port Elizabeth,-33.9180,25.5701,78.10,88,0,0.00,ZA,1643697877
305,Ibrā’,22.6906,58.5334,79.65,22,0,4.09,OM,1643697882
316,Puerto Escondido,15.8500,-97.0667,74.64,81,0,7.16,MX,1643697889


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Store into variable named hotel_df
hotel_df = smaller_city_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
1,Cape Town,ZA,-33.9258,18.4232,
52,Makakilo City,US,21.3469,-158.0858,
59,Kapaa,US,22.0752,-159.3190,
83,East London,ZA,-33.0153,27.9116,
129,Praia,CV,14.9215,-23.5087,
230,Keti Bandar,PK,24.1447,67.4497,
231,Pochutla,MX,15.7432,-96.4661,
296,Port Elizabeth,ZA,-33.9180,25.5701,
305,Ibrā’,OM,22.6906,58.5334,
316,Puerto Escondido,MX,15.8500,-97.0667,


In [7]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 1: Cape Town.
Closest hotel in Cape Town is Radisson Blu Hotel Waterfront, Cape Town.
--------------------Ending Search
Retrieving Results for Index 52: Makakilo City.
Closest hotel in Makakilo City is Aulani, A Disney Resort & Spa.
--------------------Ending Search
Retrieving Results for Index 59: Kapaa.
Closest hotel in Kapaa is The ISO.
--------------------Ending Search
Retrieving Results for Index 83: East London.
Closest hotel in East London is Blue Lagoon Hotel.
--------------------Ending Search
Retrieving Results for Index 129: Praia.
Closest hotel in Praia is Hotel Oasis Atlantico Praiamar.
--------------------Ending Search
Retrieving Results for Index 230: Keti Bandar.
Could not find result
-------------------------


NameError: name 'time' is not defined

In [8]:
 #Display the hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
1,Cape Town,ZA,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
52,Makakilo City,US,21.3469,-158.0858,"Aulani, A Disney Resort & Spa"
59,Kapaa,US,22.0752,-159.3190,The ISO
83,East London,ZA,-33.0153,27.9116,Blue Lagoon Hotel
129,Praia,CV,14.9215,-23.5087,Hotel Oasis Atlantico Praiamar
230,Keti Bandar,PK,24.1447,67.4497,
231,Pochutla,MX,15.7432,-96.4661,
296,Port Elizabeth,ZA,-33.9180,25.5701,
305,Ibrā’,OM,22.6906,58.5334,
316,Puerto Escondido,MX,15.8500,-97.0667,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig



Figure(layout=FigureLayout(height='420px'))